The purpose of this Jupyter notebook is to demonstrate connecting from a Jupyter notebook running in a container to a postgres database running in another container.

## Postgres Installation

The Postgres database is run in a separate linked container, that by default named `birth_db`. This container uses the `birth_db` image, which is built from `birth_db/Dockerfile`. That image is based on the lastest [postgres](https://hub.docker.com/_/postgres/) image. Additionally, it creates the `birth_db` database with a single table named `birth_data`, and populates `birth_data` table with the rows in `births2012_downsampled.csv`.


The default postgres user in Docker is `postgres`, but that can be changed with the `POSTGRES_USER` environment variable. 

The password for the postgres user is set by the environment variable `POSTGRES_PASSWORD`, with the defalt set in `birth_db/Dockerfile`. Obviously, the way I am doing it here is insecure and really stupid, so I need to figure out how people handle this in production. The Docker [Postgres](https://hub.docker.com/_/postgres/) docs mention something about referencing files for different environment variables.

How to reference the hostnames of linked containers took me a while to figure out and I finally found the docs [here](https://docs.docker.com/compose/networking/).

In [1]:
from sqlalchemy import create_engine

In [2]:
# Define a database name
dbname = 'birth_db'
username = 'postgres' 
password = 'mysecretpassword'
hostname = 'birth_db'
port = '5432'
db_uri = f"postgres://{username}:{password}@{hostname}:{port}/{dbname}"
engine = create_engine(db_uri)

The Postgres container has already created and populated the `birth_data` table.

In [3]:
engine.table_names()

['birth_data']

# Explore with Pandas

In [6]:
import pandas as pd

Read birth data using `pd.read_sql_table`:

In [7]:
birth_data = pd.read_sql_table("birth_data", engine)
birth_data.head()

,index,alcohol_use,anencephaly,attendant,birth_loc_type,birth_month,birth_state,birth_weight,birth_year,cigarette_use,...,mother_state,population,pregnancy_weight,resident,revision,spina_bifida,table,timestamp,uses_tobacco,weight_gain
0,0,NaN,NaN,MD,NaN,Jan,NaN,4500.0,2012,NaN,...,NaN,NaN,NaN,Resident,S,NaN,births12.txt,1325405431,NaN,40.0
1,1,NaN,NaN,MD,NaN,Jan,NaN,4500.0,2012,NaN,...,NaN,NaN,NaN,Intra-State/Territor Non-resident (diff county),S,NaN,births12.txt,1325882986,NaN,49.0
2,2,NaN,NaN,MD,NaN,Jan,NaN,2500.0,2012,NaN,...,NaN,NaN,NaN,Resident,S,NaN,births12.txt,1326367089,NaN,30.0
3,3,NaN,NaN,MD,NaN,Jan,NaN,3000.0,2012,NaN,...,NaN,NaN,NaN,Resident,S,NaN,births12.txt,1326838810,NaN,15.0
4,4,NaN,NaN,MD,NaN,Jan,NaN,4000.0,2012,NaN,...,NaN,NaN,NaN,Intra-State/Territor Non-resident (diff county),S,NaN,births12.txt,1327339729,NaN,48.0


Read birth data using `pd.read_sql` with SQL query

In [8]:
pd.read_sql("SELECT * FROM birth_data LIMIT 10", engine)

,index,alcohol_use,anencephaly,attendant,birth_loc_type,birth_month,birth_state,birth_weight,birth_year,cigarette_use,...,mother_state,population,pregnancy_weight,resident,revision,spina_bifida,table,timestamp,uses_tobacco,weight_gain
0,0,None,None,MD,None,Jan,None,4500.0,2012,None,...,None,None,None,Resident,S,None,births12.txt,1325405431,None,40.0
1,1,None,None,MD,None,Jan,None,4500.0,2012,None,...,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1325882986,None,49.0
2,2,None,None,MD,None,Jan,None,2500.0,2012,None,...,None,None,None,Resident,S,None,births12.txt,1326367089,None,30.0
3,3,None,None,MD,None,Jan,None,3000.0,2012,None,...,None,None,None,Resident,S,None,births12.txt,1326838810,None,15.0
4,4,None,None,MD,None,Jan,None,4000.0,2012,None,...,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1327339729,None,48.0
5,5,None,None,Certified Nurse or Midwife,None,Jan,None,8165.0,2012,None,...,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1327819183,None,39.0
6,6,None,None,MD,None,Feb,None,4500.0,2012,None,...,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1328297029,None,51.0
7,7,None,None,MD,None,Feb,None,4000.0,2012,None,...,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1328741505,None,40.0
8,8,None,None,MD,None,Feb,None,5000.0,2012,None,...,None,None,None,Resident,S,None,births12.txt,1329199307,None,50.0
9,9,None,None,MD,None,Feb,None,4500.0,2012,None,...,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1329642398,None,34.0


# Explore with SQLAlchemy

In [92]:
from sqlalchemy import select, MetaData, Table

In [93]:
from sqlalchemy import select

# Reflect census table via engine
metadata = MetaData()
birth_data = Table('birth_data', metadata, autoload=True, autoload_with=engine)

In [94]:
print(repr(metadata.tables['birth_data']))

Table('birth_data', MetaData(bind=None), Column('index', BIGINT(), table=<birth_data>), Column('Unnamed: 0', BIGINT(), table=<birth_data>), Column('alcohol_use', DOUBLE_PRECISION(precision=53), table=<birth_data>), Column('anencephaly', DOUBLE_PRECISION(precision=53), table=<birth_data>), Column('attendant', TEXT(), table=<birth_data>), Column('birth_loc_type', DOUBLE_PRECISION(precision=53), table=<birth_data>), Column('birth_month', TEXT(), table=<birth_data>), Column('birth_state', DOUBLE_PRECISION(precision=53), table=<birth_data>), Column('birth_weight', DOUBLE_PRECISION(precision=53), table=<birth_data>), Column('birth_year', BIGINT(), table=<birth_data>), Column('cigarette_use', DOUBLE_PRECISION(precision=53), table=<birth_data>), Column('cigarettes_per_day', DOUBLE_PRECISION(precision=53), table=<birth_data>), Column('cigarettes_trimester1', DOUBLE_PRECISION(precision=53), table=<birth_data>), Column('cigarettes_trimester2', DOUBLE_PRECISION(precision=53), table=<birth_data>), 

In [95]:
print(birth_data.columns.keys())

['index', 'Unnamed: 0', 'alcohol_use', 'anencephaly', 'attendant', 'birth_loc_type', 'birth_month', 'birth_state', 'birth_weight', 'birth_year', 'cigarette_use', 'cigarettes_per_day', 'cigarettes_trimester1', 'cigarettes_trimester2', 'cigarettes_trimester3', 'day', 'delivery_method', 'downs syndrome', 'drinks_per_week', 'father_age', 'father_race', 'gestation_weeks', 'infant_sex', 'mother_age', 'mother_birth_country', 'mother_birth_state', 'mother_education', 'mother_marital_status', 'mother_race', 'mother_state', 'population', 'pregnancy_weight', 'resident', 'revision', 'spina_bifida', 'table', 'timestamp', 'uses_tobacco', 'weight_gain']


Create a connection to use for queries.

In [96]:
connection = engine.connect()

In [97]:
stmt = "SELECT * FROM birth_data"

results = connection.execute(stmt).fetchall()

First results

In [98]:
results[:1]

[(0, 0, None, None, 'MD', None, 'Jan', None, 4500.0, 2012, None, None, None, None, None, 'Fri', 'Vaginal', None, None, None, None, 40.0, 'M', 32, None, None, None, 'Yes', 'White', None, None, None, 'Resident', 'S', None, 'births12.txt', 1325405431, None, 40.0)]

Alternatively use `fetchone()` method.

In [99]:
connection.execute(stmt).fetchone()

(0, 0, None, None, 'MD', None, 'Jan', None, 4500.0, 2012, None, None, None, None, None, 'Fri', 'Vaginal', None, None, None, None, 40.0, 'M', 32, None, None, None, 'Yes', 'White', None, None, None, 'Resident', 'S', None, 'births12.txt', 1325405431, None, 40.0)

The following examples use the SQLAlchemy expression language to generate various statements.

In [104]:
stmt = select([birth_data])
type(stmt)

sqlalchemy.sql.selectable.Select

Printing this object simply prints a SQL query. The advantage of SQLAlchemy is that it coverts the statement to the speicfic query language.

In [106]:
print(stmt)

SELECT birth_data.index, birth_data."Unnamed: 0", birth_data.alcohol_use, birth_data.anencephaly, birth_data.attendant, birth_data.birth_loc_type, birth_data.birth_month, birth_data.birth_state, birth_data.birth_weight, birth_data.birth_year, birth_data.cigarette_use, birth_data.cigarettes_per_day, birth_data.cigarettes_trimester1, birth_data.cigarettes_trimester2, birth_data.cigarettes_trimester3, birth_data.day, birth_data.delivery_method, birth_data."downs syndrome", birth_data.drinks_per_week, birth_data.father_age, birth_data.father_race, birth_data.gestation_weeks, birth_data.infant_sex, birth_data.mother_age, birth_data.mother_birth_country, birth_data.mother_birth_state, birth_data.mother_education, birth_data.mother_marital_status, birth_data.mother_race, birth_data.mother_state, birth_data.population, birth_data.pregnancy_weight, birth_data.resident, birth_data.revision, birth_data.spina_bifida, birth_data."table", birth_data.timestamp, birth_data.uses_tobacco, birth_data.wei

In [107]:
connection.execute(stmt).fetchone()

(0, 0, None, None, 'MD', None, 'Jan', None, 4500.0, 2012, None, None, None, None, None, 'Fri', 'Vaginal', None, None, None, None, 40.0, 'M', 32, None, None, None, 'Yes', 'White', None, None, None, 'Resident', 'S', None, 'births12.txt', 1325405431, None, 40.0)

In [108]:
from itertools import islice
for res in islice(results, 10):
    print((res.index, res.alcohol_use))

(0, None)
(1, None)
(2, None)
(3, None)
(4, None)
(5, None)
(6, None)
(7, None)
(8, None)
(9, None)


```
SELECT * FROM birth_data_table WHERE infant_sex='M';
```

In [109]:
stmt = select([birth_data]).\
    where(birth_data.columns.infant_sex == "M").\
    limit(3)

In [110]:
connection.execute(stmt).fetchall()

[(0, 0, None, None, 'MD', None, 'Jan', None, 4500.0, 2012, None, None, None, None, None, 'Fri', 'Vaginal', None, None, None, None, 40.0, 'M', 32, None, None, None, 'Yes', 'White', None, None, None, 'Resident', 'S', None, 'births12.txt', 1325405431, None, 40.0),
 (2, 2, None, None, 'MD', None, 'Jan', None, 2500.0, 2012, None, None, None, None, None, 'Sat', 'Cesarean', None, None, None, None, 36.0, 'M', 28, None, None, None, 'No', 'Black', None, None, None, 'Resident', 'S', None, 'births12.txt', 1326367089, None, 30.0),
 (3, 3, None, None, 'MD', None, 'Jan', None, 3000.0, 2012, None, None, None, None, None, 'Sun', 'Vaginal', None, None, None, None, 40.0, 'M', 28, None, None, None, 'No', 'Black', None, None, None, 'Resident', 'S', None, 'births12.txt', 1326838810, None, 15.0)]

```
SELECT COUNT(infant_sex) FROM birth_data_table WHERE infant_sex='M';
```

In [111]:
from sqlalchemy import func

In [112]:
stmt = select([func.count(birth_data.columns.infant_sex)]).\
    where(birth_data.columns.infant_sex == "M").\
    limit(3)

Since this returns one value, I can return it as a scalar.

In [113]:
connection.execute(stmt).scalar()

1394

```
SELECT COUNT(gestation_weeks), infant_sex FROM birth_data_table WHERE infant_sex = 'M' GROUP BY gestation_weeks, infant_sex;
```

Statements can also be executed by `pandas.read_sql`.

In [80]:
stmt = select([birth_data.columns.gestation_weeks, 
               func.count(birth_data.columns.gestation_weeks).label("count")]).\
    where(birth_data.columns.infant_sex == "M").\
    group_by(birth_data.columns.gestation_weeks)
pd.read_sql(stmt, connection)


,gestation_weeks,count
0,31.0,14
1,33.0,18
2,38.0,365
3,27.0,3
4,41.0,99
5,42.0,83
6,20.0,1
7,36.0,130
8,40.0,275
9,39.0,406


Loop of and print parts of the results

# Creating the Database After Starting

I could have avoided creating the database, createing the `birth_data` table, or loading the csv into the table from here.

In [81]:
from sqlalchemy_utils import create_database, drop_database

In [82]:
from sqlalchemy.exc import ProgrammingError
try:
    drop_database(db_uri)
except ProgrammingError:
    pass

In [83]:
create_database(db_uri)

In [84]:
# need new engine since old db destroyed
engine = create_engine(db_uri)

In [85]:
birth_data = pd.read_csv("births2012_downsampled.csv")

In [86]:
birth_data.to_sql("birth_data", db_uri)

# Pandas and SQL

This is a cool trick I want to remember. I used SQLAlachemy/pandas to write the `CREATE TABLE` statement. Pandas does this in the background with `to_sql`, but the methods it uses are private and took a little digging.

In [87]:
from pandas.io.sql import SQLDatabase, SQLTable

In [13]:
db = SQLDatabase(engine)

In [22]:
birth_data = birth_data.reset_index()

In [23]:
print(db._create_sql_schema(table_name='birth_data_table', frame=birth_data, keys=['index']))


CREATE TABLE birth_data_table (
	index BIGSERIAL NOT NULL, 
	alcohol_use FLOAT(53), 
	anencephaly FLOAT(53), 
	attendant TEXT, 
	birth_loc_type FLOAT(53), 
	birth_month TEXT, 
	birth_state FLOAT(53), 
	birth_weight FLOAT(53), 
	birth_year BIGINT, 
	cigarette_use FLOAT(53), 
	cigarettes_per_day FLOAT(53), 
	cigarettes_trimester1 FLOAT(53), 
	cigarettes_trimester2 FLOAT(53), 
	cigarettes_trimester3 FLOAT(53), 
	day TEXT, 
	delivery_method TEXT, 
	"downs syndrome" FLOAT(53), 
	drinks_per_week FLOAT(53), 
	father_age FLOAT(53), 
	father_race TEXT, 
	gestation_weeks FLOAT(53), 
	infant_sex TEXT, 
	mother_age BIGINT, 
	mother_birth_country FLOAT(53), 
	mother_birth_state FLOAT(53), 
	mother_education TEXT, 
	mother_marital_status TEXT, 
	mother_race TEXT, 
	mother_state FLOAT(53), 
	population FLOAT(53), 
	pregnancy_weight FLOAT(53), 
	resident TEXT, 
	revision TEXT, 
	spina_bifida FLOAT(53), 
	"table" TEXT, 
	timestamp BIGINT, 
	uses_tobacco FLOAT(53), 
	weight_gain FLOAT(53), 
	CONSTRAINT